In [14]:
import cv2
import os
import numpy as np


In [15]:
subjects = [None,"Mark Zuckerberg","Akshay Kumar"]

In [16]:
def detect_face(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.01, minNeighbors=1);
    
    if(len(faces) == 0):
        return None,None
    
    x, y, w, h = faces[0]
    return gray[y:y+w,x:x+h], faces[0]

In [26]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue;
        label = int(dir_name.replace("s",""))
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue;
                
            image_path = subject_dir_path + "/" + image_name
            image = cv2.imread(image_path)
            face, rect = detect_face(image)
            if face is not None:
                print(image_name, "Face detected")
                faces.append(face)
                labels.append(label)
            else:
                print(image_name, "Face not detected")
            cv2.destroyAllWindows()
            cv2.waitKey(1)
        cv2.destroyAllWindows()
        
    return faces, labels

In [28]:
print("Preparing Data...")
faces, labels = prepare_training_data("training_face")
print("Data prepared")

print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing Data...
mark1.jpeg Face detected
mark3.webp Face detected
mark.jpeg Face detected
akshay_1.jpg Face detected
akshay_kumar.jpg Face detected
Akshay.jpg Face detected
Data prepared
Total faces:  6
Total labels:  6


In [30]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [31]:
face_recognizer.train(faces, np.array(labels))

In [32]:
def predict(test_img):
    img = test_img.copy()
    face,rect = detect_face(img)
    
    if face is None:
        return False, "No Face detected"
    
    label = face_recognizer.predict(face)
    label_text = subjects[label[0]]
    
    return True, label_text

In [33]:
print("Predicting images...")
test_img1 = cv2.imread("testing_face/test1.jpg")
detected1, predicted_img1 = predict(test_img1)
if detected1:
    print('Image 1', predicted_img1)
else:
    print(predicted_img1)

print('----------------')

test_img2 = cv2.imread("testing_face/test2.jpg")
detected2, predicted_img2 = predict(test_img2)
if detected2:
    print('Image 2', predicted_img2)
else:
    print(predicted_img2)
    
print("Prediction complete")


    


Predicting images...
Image 1 Akshay Kumar
----------------
Image 2 Akshay Kumar
Prediction complete
